# Loading pretrained model

https://www.reddit.com/r/MediaSynthesis/comments/gum6f1/tfdne_edit_furry_faces_inbrowser_google_colab/

In [ ]:
# !git clone https://github.com/shawwn/stylegan2 -b estimator /content/stylegan2

In [16]:
import gdown
!wget  -O /content/network-e621.pkl https://thisfursonadoesnotexist.com/model/network-e621-r-512-3194880.pkl
# !wget  -O /content/directions.zip https://thisfursonadoesnotexist.com/directions.zip

--2022-04-28 18:44:36--  https://thisfursonadoesnotexist.com/model/network-e621-r-512-3194880.pkl
Resolving thisfursonadoesnotexist.com (thisfursonadoesnotexist.com)... 172.67.159.181, 104.21.14.150, 2606:4700:3034::ac43:9fb5, ...
Connecting to thisfursonadoesnotexist.com (thisfursonadoesnotexist.com)|172.67.159.181|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 364035409 (347M) [application/octet-stream]
Saving to: ‘/content/network-e621.pkl’

/content/network-e6 100%[===================>] 347.17M  7.21MB/s    in 48s     

2022-04-28 18:45:25 (7.25 MB/s) - ‘/content/network-e621.pkl’ saved [364035409/364035409]



In [19]:
%tensorflow_version 1.x
%cd /content/stylegan2

/content/stylegan2


In [23]:
import os
import pickle
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import scipy
import tensorflow as tf
# import tflex

In [21]:
if 'COLAB_TPU_ADDR' in os.environ:
    os.environ['TPU_NAME'] = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    os.environ['NOISY'] = '1'

In [24]:
tflib.init_tf()
sess = tf.get_default_session()
sess.list_devices()
# cores = tflex.get_cores()
# tflex.set_override_cores(cores)
_G, _D, Gs = pickle.load(open("/content/network-e621.pkl", "rb"))


# _G = Instantaneous snapshot of the generator. Mainly useful for resuming a previous training run.
# _D = Instantaneous snapshot of the discriminator. Mainly useful for resuming a previous training run.
# Gs = Long-term average of the generator. Yields higher-quality results than the instantaneous snapshot.

Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Compiling... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Compiling... Loading... Done.


In [ ]:
 def generate_image_from_w(w, truncation_psi):
    with tflex.device('/gpu:0'):
        #_G, _D, Gs = pickle.load(open("/content/network-e621.pkl", "rb"))
        noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
        Gs_kwargs = dnnlib.EasyDict()
        Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
        Gs_kwargs.randomize_noise = False
        if truncation_psi is not None:
            Gs_kwargs.truncation_psi = truncation_psi
        synthesis_kwargs = dict(output_transform=Gs_kwargs.output_transform, truncation_psi=truncation_psi, minibatch_size=1)
        images = Gs.components.synthesis.run(w, randomize_noise=False, **synthesis_kwargs)
        #PIL.Image.fromarray(images[0], 'RGB').save('seed%04d.png' % seed)
        display(PIL.Image.fromarray(images[0], 'RGB'))



---
# Loading data into tfrecords





---

**Loading tf records**

https://www.kaggle.com/datasets/brendanartley/anime-faces-tfrecords

First upload kaggle.json. May have to pause a min after uploading before running dataset download code.

In [ ]:
# %%capture
# ! pip install kaggle
# ! mkdir ~/.kaggle
# ! cp kaggle.json ~/.kaggle/
# ! chmod 600 ~/.kaggle/kaggle.json
# ! kaggle datasets download brendanartley/anime-faces-tfrecords
# ! unzip anime-faces-tfrecords.zip -d ./dataset



---

**Loading low-res faces then upscaling**


https://www.kaggle.com/datasets/splcher/animefacedataset/code

In [ ]:
# %%capture
# ! pip install kaggle
# ! mkdir ~/.kaggle
# ! cp kaggle.json ~/.kaggle/
# ! chmod 600 ~/.kaggle/kaggle.json
# ! kaggle datasets download splcher/animefacedataset
# ! unzip animefacedataset.zip 

In [ ]:
import os 
import numpy as np
import PIL.Image

origin_dir = './images'
image_names = [files for root, dirs, files in os.walk(origin_dir)][0]
for index, image_name in enumerate(image_names):
    image_path = os.path.join(origin_dir, image_name)
    img = np.asarray(PIL.Image.open(image_path))
    width, height = PIL.Image.open(image_path).size
    print(width, height)
    # img = img.reshape(1, 3, 512, 512)

We see all images are different sizes. We need to upscale them, then reshape them, else they cannot be transformed into tfrecord.

In [ ]:
!cat /usr/local/cuda/version.txt
!wget http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/cuda-repo-ubuntu1604_9.0.176-1_amd64.deb
!sudo dpkg -i cuda-repo-ubuntu1604_9.0.176-1_amd64.deb
!apt-get install libvulkan-dev
!apt-get update

!%cd /content/
!git clone https://github.com/nihui/waifu2x-ncnn-vulkan.git
!cd waifu2x-ncnn-vulkan/
!git submodule update --init --recursive
!wget https://github.com/nihui/waifu2x-ncnn-vulkan/releases/download/20200606/waifu2x-ncnn-vulkan-20200606-linux.zip
!unzip waifu2x-ncnn-vulkan-20200606-linux.zip
%cd waifu2x-ncnn-vulkan-20200606-linux



In [ ]:
# upload all files first ->
%mkdir 2x
# !for img in *.??g; do ./waifu2x-ncnn-vulkan -i $img -o 2x/${img%.*}_2x.png; done
!for img in /content/images; do ./waifu2x-ncnn-vulkan -i $img -o 2x/${img%.*}_2x.png; done


# copying in gdrive
#!gsutil -m cp -r hand_tuned_larger/ '/content/drive/My Drive/twist_moe/hand_tuned_larger_2x/'

mkdir: cannot create directory ‘2x’: File exists
./waifu2x-ncnn-vulkan: error while loading shared libraries: libvulkan.so.1: cannot open shared object file: No such file or directory




---

**Loading a subset of high-res faces**

https://www.kaggle.com/datasets/subinium/highresolution-anime-face-dataset-512x512

In [4]:
%%capture
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download subinium/highresolution-anime-face-dataset-512x512
# ! unzip highresolution-anime-face-dataset-512x512.zip 

Unzip only 5000 files from zip

https://stackoverflow.com/questions/22243031/unzip-only-limited-number-of-files-in-linux

In [ ]:
!unzip -Z1 highresolution-anime-face-dataset-512x512.zip | head -5000 | sed 's| |\\ |g' | xargs unzip highresolution-anime-face-dataset-512x512.zip

In [ ]:
!pip install numpy==1.19.5

%tensorflow_version 1.x
import tensorflow as tf

# Download the code
!git clone https://github.com/ZKTKZ/stylegan2.git
# %cd stylegan2
# !nvcc test_nvcc.cu -o test_nvcc -run
!nvcc /content/stylegan2/test_nvcc.cu -o /content/stylegan2/test_nvcc -run

print('Tensorflow version: {}'.format(tf.__version__) )
!nvidia-smi -L
print('GPU Identified at: {}'.format(tf.test.gpu_device_name()))

!pip install tensorboard

In [15]:
# convert images to tfrecord

!python /content/stylegan2/dataset_tool.py create_from_images ./training_data ./portraits 

Loading images from "./portraits"
detected 5000 images ...
Creating dataset "./training_data"
Adding the images to tfrecords ...
/content/stylegan2/dataset_tool.py:154: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  bytes_list=tf.train.BytesList(value=[quant.tostring()])
Added 5000 images.


https://www.tensorflow.org/tutorials/load_data/tfrecord



---

**Download high-res directly from drive**


In [ ]:
# import gdown
# from google_drive_downloader import GoogleDriveDownloader as gdd

# model_id = '' #get from 'copy link', after /d/ and before /view
# network_pkl = '/content/models/model_%s.pkl' % model_id#(hashlib.md5(model_id.encode()).hexdigest())
# gdd.download_file_from_google_drive(file_id=model_id, dest_path=network_pkl)

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')  #brings google drive files to be accessible under /content/

Mounted at /content/drive


In [ ]:
!unzip -u "/content/drive/My Drive/high_res_anime_faces.zip" -d ./data

In [ ]:
# cannot run this on .zip
!python /content/stylegan2/dataset_tool.py create_from_images \
    "/content/drive/My Drive/high_res_anime_faces_tfrecords" \
    "/content/data"



---
# Training




https://colab.research.google.com/github/ZKTKZ/thdne/blob/master/StyleGAN2_Tazik_25GB_RAM.ipynb#scrollTo=vmeWpLdZdhq2

In [27]:
!python /content/stylegan2/run_training.py --num-gpus=1 --data-dir=/content/ --config=config-f --dataset=training_data \
--total-kimg=10000 --result-dir="/content/drive/My Drive/twist_moe/results/" --resume-pkl='../network-e621.pkl'


Local submit - run_dir: /content/drive/My Drive/twist_moe/results/00002-stylegan2-training_data-1gpu-config-f
dnnlib: Running training.training_loop.training_loop() on localhost...
Streaming data using training.dataset.TFRecordDataset...
Dataset shape = [3, 512, 512]
Dynamic range = [0, 255]
Label size    = 0
2022-04-28 18:50:33.488143: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 94371840 exceeds 10% of system memory.
Loading networks from "../network-e621.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Loading... Done.

G                             Params    OutputShape         WeightShape     
---                           ---       ---                 ---             
latents_in                    -         (?, 512)            -               
labels_in                     -         (?, 0)              -               
lod                           -   

In [ ]:
# 00002-stylegan2-training_data-1gpu-config-f
# network-snapshot-000000
!python /content/stylegan2/run_generator.py generate-images --seeds=0-100 --truncation-psi=0.7 \
--network='/content/drive/My Drive/twist_moe/results/00002-stylegan2-training_data-1gpu-config-f/network-snapshot-000000.pkl'
#%cp -av /content/stylegan2/results/00000-generate-images /content/drive/'My Drive'/twist_moe/seeds-1.0


In [ ]:
import os
import pickle
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import scipy
import math
import moviepy.editor
from numpy import linalg


def main():
    tflib.init_tf()
    _G, _D, Gs = pickle.load(open("/content/drive/My Drive/twist_moe/results/00002-stylegan2-training_data-1gpu-config-f/network-snapshot-000000.pkl", "rb"))

    rnd = np.random
    latents_a = rnd.randn(1, Gs.input_shape[1])
    latents_b = rnd.randn(1, Gs.input_shape[1])
    latents_c = rnd.randn(1, Gs.input_shape[1])

    def circ_generator(latents_interpolate):
        radius = 40.0

        latents_axis_x = (latents_a - latents_b).flatten() / linalg.norm(latents_a - latents_b)
        latents_axis_y = (latents_a - latents_c).flatten() / linalg.norm(latents_a - latents_c)

        latents_x = math.sin(math.pi * 2.0 * latents_interpolate) * radius
        latents_y = math.cos(math.pi * 2.0 * latents_interpolate) * radius

        latents = latents_a + latents_x * latents_axis_x + latents_y * latents_axis_y
        return latents

    def mse(x, y):
        return (np.square(x - y)).mean()

    def generate_from_generator_adaptive(gen_func):
        max_step = 1.0
        current_pos = 0.0

        change_min = 10.0
        change_max = 11.0

        fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)

        current_latent = gen_func(current_pos)
        current_image = Gs.run(current_latent, None, truncation_psi=0.7, randomize_noise=False, output_transform=fmt)[0]
        array_list = []

        video_length = 1.0
        while(current_pos < video_length):
            array_list.append(current_image)

            lower = current_pos
            upper = current_pos + max_step
            current_pos = (upper + lower) / 2.0

            current_latent = gen_func(current_pos)
            current_image = images = Gs.run(current_latent, None, truncation_psi=0.7, randomize_noise=False, output_transform=fmt)[0]
            current_mse = mse(array_list[-1], current_image)

            while current_mse < change_min or current_mse > change_max:
                if current_mse < change_min:
                    lower = current_pos
                    current_pos = (upper + lower) / 2.0

                if current_mse > change_max:
                    upper = current_pos
                    current_pos = (upper + lower) / 2.0


                current_latent = gen_func(current_pos)
                current_image = images = Gs.run(current_latent, None, truncation_psi=0.7, randomize_noise=False, output_transform=fmt)[0]
                current_mse = mse(array_list[-1], current_image)
            print(current_pos, current_mse)
        return array_list

    frames = generate_from_generator_adaptive(circ_generator)
    frames = moviepy.editor.ImageSequenceClip(frames, fps=30)

    # Generate video.
    mp4_file = 'circular.mp4'
    mp4_codec = 'libx264'
    mp4_bitrate = '3M'
    mp4_fps = 20

    frames.write_videofile(mp4_file, fps=mp4_fps, codec=mp4_codec, bitrate=mp4_bitrate)

if __name__ == "__main__":
    main()

In [38]:
#https://stackoverflow.com/a/57378660/8773953
!pip install -U kora
from kora.drive import upload_public
url = upload_public('/content/stylegan2/circular.mp4')

from IPython.display import HTML
HTML(f"""<video src={url} width=500 controls/>""")

In [ ]:
!zip -r /content/anime_faces_furry_pretr.zip /content/drive/My\ Drive/twist_moe